In [2]:
import pandas as pd

In [3]:
uhauldata = pd.read_csv("../data/uhauloutput-35cities.csv")

In [4]:
incomedata = pd.read_csv("../data/income32.csv")

In [6]:
uhauldata_filtered = uhauldata[
    (uhauldata['Pickup Location'].isin(incomedata["City"].unique())) &
    (uhauldata['Dropoff Location'].isin(incomedata["City"].unique()))
]

In [7]:
uhauldata_filtered

,Pickup Location,Dropoff Location,Rate,Truck Type,Moving Type,Date
0,"Atlanta, GA","Boston, MA","$2,111.00",10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
1,"Atlanta, GA","Boston, MA","$2,223.00",15' Truck,1 Bedroom Home to 2 Bedroom Apt.,09/28/2024
2,"Atlanta, GA","Boston, MA","$2,443.00",20' Truck,2 Bedroom Home to 3 Bedroom Apt.,09/28/2024
3,"Atlanta, GA","Boston, MA","$2,779.00",26' Truck,3 Bedroom Home to 4 Bedroom Home,09/28/2024
8,"Atlanta, GA","Charlotte, NC",$429.00,10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
...,...,...,...,...,...,...
4732,"Wichita, KS","Tucson, AZ","$2,023.00",26' Truck,3 Bedroom Home to 4 Bedroom Home,09/28/2024
4733,"Wichita, KS","Tulsa, OK",$285.00,10' Truck,Studio to 1 Bedroom Apt.,09/28/2024
4734,"Wichita, KS","Tulsa, OK",$300.00,15' Truck,1 Bedroom Home to 2 Bedroom Apt.,09/28/2024
4735,"Wichita, KS","Tulsa, OK",$331.00,20' Truck,2 Bedroom Home to 3 Bedroom Apt.,09/28/2024


(32,)